## 2. go.mod 檔案

`go.mod` 檔案是 Go 模組的核心配置檔案，類似於其他語言的依賴管理檔案（如 Node.js 的 package.json 或 Python 的 requirements.txt）。

### go.mod 的基本結構：
- **module**: 定義模組的匯入路徑
- **go**: 指定 Go 版本要求
- **require**: 列出直接依賴
- **exclude**: 排除特定版本
- **replace**: 替換依賴的來源
- **retract**: 撤回已發布的版本

#### 版本語義化：
Go 使用語義化版本控制 (Semantic Versioning)：
- **主版本.次版本.修訂版本** (如 v1.2.3)
- **主版本變更**: 破壞性變更
- **次版本變更**: 新功能，向後相容
- **修訂版本**: 錯誤修復

#### go.mod 指令：
- `go mod init`: 初始化新模組
- `go mod tidy`: 清理和更新依賴
- `go mod download`: 下載依賴到本地快取
- `go mod verify`: 驗證依賴的完整性"

# 第九章 模組、程式包與匯入 (Modules, Packages and Imports)

## 章節概述

Go 的模組系統是管理依賴關係和程式碼組織的核心機制。從 Go 1.11 開始引入模組系統，徹底改變了 Go 專案的組織方式和依賴管理。本章將深入探討如何有效地組織和管理 Go 程式碼。

### 本章重點內容
- **版本庫、模組與程式包的關係**: 理解三者的概念和層級關係
- **go.mod 檔案**: 模組的核心配置檔案
- **建構程式包**: 如何設計和實作程式包
- **匯入與匯出**: 控制程式包的可見性
- **程式包命名**: 最佳實踐和慣例
- **模組組織**: 如何合理組織大型專案
- **internal 程式包**: 限制程式包的可見範圍
- **程式包註解與文件**: 使用 godoc 產生文件
- **依賴管理**: 版本控制和更新策略
- **vendoring**: 本地依賴管理

### Go 模組系統的核心概念
1. **模組 (Module)**: 相關程式包的集合，是版本控制的基本單位
2. **程式包 (Package)**: Go 程式碼的組織單位，對應一個目錄
3. **版本庫 (Repository)**: 儲存程式碼的地方，通常包含一個或多個模組

In [ ]:
// 這個程式展示模組系統的基本概念和組織結構
package main

import (
    "fmt"
    "go/ast"
    "go/parser"
    "go/token"
    "log"
    "os"
    "path/filepath"
    "strings"
)

// 展示 Go 專案結構分析工具
func main() {
    fmt.Println("=== Go 模組系統概念示範 ===")
    
    // 1. 展示模組的基本概念
    demonstrateModuleConcepts()
    
    // 2. 分析當前專案的程式包結構
    fmt.Println("\n=== 分析專案結構 ===")
    analyzeProjectStructure(".")
    
    // 3. 展示匯入路徑的概念
    fmt.Println("\n=== 匯入路徑示範 ===")
    demonstrateImportPaths()
}

// 展示模組系統的核心概念
func demonstrateModuleConcepts() {
    fmt.Println("\n--- 模組系統核心概念 ---")
    
    // 模組路徑範例
    moduleExamples := []struct {
        name        string
        modulePath  string
        description string
    }{
        {
            name:        "標準庫",
            modulePath:  "std",
            description: "Go 官方標準庫，包含 fmt, os, net/http 等",
        },
        {
            name:        "第三方庫",
            modulePath:  "github.com/gin-gonic/gin",
            description: "流行的 Web 框架，託管在 GitHub",
        },
        {
            name:        "企業內部",
            modulePath:  "company.com/internal/auth",
            description: "企業內部的認證模組",
        },
        {
            name:        "個人專案",
            modulePath:  "example.com/myproject",
            description: "個人開發的專案模組",
        },
    }
    
    for _, example := range moduleExamples {
        fmt.Printf("  %s:\n", example.name)
        fmt.Printf("    模組路徑: %s\n", example.modulePath)
        fmt.Printf("    說明: %s\n\n", example.description)
    }
}

// 分析專案結構（簡化版）
func analyzeProjectStructure(rootPath string) {
    packages := make(map[string][]string)
    
    // 遍歷專案目錄
    err := filepath.Walk(rootPath, func(path string, info os.FileInfo, err error) error {
        if err != nil {
            return err
        }
        
        // 只分析 .go 檔案
        if !strings.HasSuffix(info.Name(), ".go") {
            return nil
        }
        
        // 跳過測試檔案和隱藏目錄
        if strings.HasSuffix(info.Name(), "_test.go") || strings.Contains(path, "/.git/") {
            return nil
        }
        
        // 解析 Go 檔案取得程式包名稱
        packageName, err := extractPackageName(path)
        if err != nil {
            return nil // 忽略無法解析的檔案
        }
        
        dir := filepath.Dir(path)
        packages[dir] = append(packages[dir], packageName)
        
        return nil
    })
    
    if err != nil {
        fmt.Printf("分析專案結構時發生錯誤: %v\n", err)
        return
    }
    
    // 顯示分析結果
    fmt.Println("  發現的程式包:")
    for dir, pkgNames := range packages {
        if len(pkgNames) > 0 {
            uniquePkgs := uniqueStrings(pkgNames)
            fmt.Printf("    %s: %v\n", dir, uniquePkgs)
        }
    }
}

// 從 Go 檔案中提取程式包名稱
func extractPackageName(filename string) (string, error) {
    fset := token.NewFileSet()
    file, err := parser.ParseFile(fset, filename, nil, parser.PackageClauseOnly)
    if err != nil {
        return "", err
    }
    
    return file.Name.Name, nil
}

// 展示匯入路徑的概念
func demonstrateImportPaths() {
    importExamples := []struct {
        importPath  string
        packageName string
        usage       string
        category    string
    }{
        {
            importPath:  "fmt",
            packageName: "fmt",
            usage:       "fmt.Println(\"Hello\")",
            category:    "標準庫",
        },
        {
            importPath:  "net/http",
            packageName: "http",
            usage:       "http.Get(\"https://example.com\")",
            category:    "標準庫子包",
        },
        {
            importPath:  "github.com/gorilla/mux",
            packageName: "mux",
            usage:       "mux.NewRouter()",
            category:    "第三方庫",
        },
        {
            importPath:  "myproject/internal/auth",
            packageName: "auth",
            usage:       "auth.ValidateToken(token)",
            category:    "內部程式包",
        },
    }
    
    for _, example := range importExamples {
        fmt.Printf("  %s:\n", example.category)
        fmt.Printf("    匯入路徑: %s\n", example.importPath)
        fmt.Printf("    程式包名: %s\n", example.packageName)
        fmt.Printf("    使用方式: %s\n\n", example.usage)
    }
}

// 輔助函數：取得字串陣列的唯一值
func uniqueStrings(strings []string) []string {
    keys := make(map[string]bool)
    var unique []string
    
    for _, str := range strings {
        if !keys[str] {
            keys[str] = true
            unique = append(unique, str)
        }
    }
    
    return unique
}